In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
# LDA csomagok

In [ ]:
pip install gensim==3.8.1 --user #ez a verzió támogatja a wrappers-t, kell a pyLDAvis-hez

In [ ]:
import os
from gensim.models.wrappers import LdaMallet
os.environ.update({'MALLET_HOME':r'C:/mallet/mallet-2.0.8/'})
mallet_path = r'C:/mallet/mallet-2.0.8/bin/mallet.bat'
#MALLET útvonal megadás

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import MalletCorpus
from gensim.models import CoherenceModel
import os
from os import listdir
from os.path import join, isfile
import logging
import codecs
import tqdm
import numpy as np
#szükséges csomagok behívása

In [ ]:
#végleges preprocess fájlok behívása
in_folder = 'C:/Users/zabor/Desktop/preprocess/adatok/Data/txts_final'
in_path = os.path.join(in_folder)
in_files = [f for f in os.listdir(in_path) if os.path.isfile(os.path.join(in_path, f))]

#összes fájl listába rakása
txt = []
for filename in tqdm.tqdm(in_files):
    try:
        with open(os.path.join(in_path, filename), 'r', encoding='utf-8') as infile:
            article = infile.read()
        txt.append(article)
    except Exception as e:
        print(e)

In [ ]:
txt[0]

In [ ]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"wonât", "will not", phrase)
    phrase = re.sub(r"canât", "can not", phrase)

    # general
    phrase = re.sub(r"nât", " not", phrase)
    phrase = re.sub(r"âre", " are", phrase)
    phrase = re.sub(r"âs", " is", phrase)
    phrase = re.sub(r"âd", " would", phrase)
    phrase = re.sub(r"âll", " will", phrase)
    phrase = re.sub(r"ât", " not", phrase)
    phrase = re.sub(r"âve", " have", phrase)
    phrase = re.sub(r"âm", " am", phrase)
    return phrase
#összevonások kezelése (can't -> can + not), beépített contractions.fix nem ismerte föl 'â' miatt

In [ ]:
txt2 = []
for i in txt:
    txt2.append(decontracted(i))

In [ ]:
txt2[0]

In [ ]:
txt_list = [t.split() for t in txt2]

In [ ]:
logging.basicConfig(filename="C:/Users/zabor/Desktop/LDA/gensim.log",
                   format="%(asctime)s:%(levelname)s:%(message)s",
                   level=logging.INFO)

In [ ]:
import itertools
import operator
from collections import Counter
from os import listdir, remove


##### GET WORDFREQ

in_path = 'C:/Users/zabor/Desktop/preprocess/adatok/Data/txts_final'
corpus = []
files = sorted([f for f in listdir(in_path) if isfile(join(in_path, f))])
for f in files:
    try:
        txt = codecs.open(join(in_path, f), 'r', 'utf-8').read()
        txt = decontracted(txt)
        txt = txt.strip().split()
        txt = [w.lower() for w in txt if len(w) > 3]
        corpus.append(txt)
    except Exception as e:
        print(f, e)

wordlist = list(itertools.chain(*corpus))
wfreq = Counter(wordlist)
sorted_wfreq = sorted(wfreq.items(),
                      key=operator.itemgetter(1),
                      reverse=True)

with codecs.open('C:/Users/zabor/Desktop/LDA/wfreq.tsv', 'w', 'utf-8') as f:
    for e in sorted_wfreq:
        if e[1] > 3 and e[1] < 10000:
            o = e[0] + '\t' + str(e[1]) + '\n'
            f.write(o)



##### GET DOCFREQ


wd_docfreq = {}
for d in corpus:
    d = list(set(d))
    for w in d:
        if w not in wd_docfreq.keys():
            wd_docfreq[w] = 1
        else:
            wd_docfreq[w] += 1

sorted_docfreq = sorted(wd_docfreq.items(),
                        key=operator.itemgetter(1),
                        reverse=True)

with codecs.open('C:/Users/zabor/Desktop/LDA/docfreq.tsv', 'w', 'utf-8') as f:
    for e in sorted_docfreq:
        o = e[0] + '\t' + str((e[1]/len(corpus))) + '\n'
        f.write(o)



#############################################################################
#####                          Content words                            #####
#############################################################################


content_words = []
of = codecs.open('C:/Users/zabor/Desktop/LDA/contentwords.tsv', 'w', 'utf-8')
with codecs.open('C:/Users/zabor/Desktop/LDA/docfreq.tsv', 'r', 'utf-8') as input_file:
    for l in input_file:
        wd, docfreq = l.strip().split('\t')
        docfreq = float(docfreq)
        if docfreq > 0.00010:
            content_words.append(wd)
            o = wd + '\t' + str(docfreq) + '\n'
            of.write(o)
of.close()


In [ ]:
# stopszavak
stopwords = []


# content words kimentése
out_path = 'C:/Users/zabor/Desktop/LDA/Data/txts_contwrd'
folder = [f for f in listdir(out_path) if isfile(join(out_path, f))]
if len(folder) != 0:
    for f in folder:
        remove(join(out_path, f))

in_path = 'C:/Users/zabor/Desktop/preprocess/adatok/Data/txts_final'
files = sorted([f for f in listdir(in_path) if isfile(join(in_path, f))])
file=0
for f in files:
    with codecs.open(join(out_path, files[file]), 'w', 'utf-8') as out_file:
        try:
            txt = codecs.open(join(in_path, f), 'r', 'utf-8').read()
            txt = txt.strip().split()
            for cw in txt:
                if cw in content_words and cw not in stopwords:
                    out_file.write(cw + " ")
        except Exception as e:
            print(files[file], e)
    file += 1

In [ ]:
import itertools
import matplotlib.pyplot as plt
import pandas as pd
from multiprocessing import freeze_support
from datetime import datetime
# még több csomag

In [ ]:
#############################################################################
#####                          LDA                                      #####
#############################################################################

# Save document to topic Excel files for each run?
save_document_to_topic = False

# Save visualization files for each run?
save_visualization = False

# Define in_folder for corpus files.
in_folder_corpus = 'C:/Users/zabor/Desktop/LDA/Data/txts_contwrd'
in_folder_original = 'C:/Users/zabor/Desktop/preprocess/adatok/Data/Nolink_txts'

# TODO: egy közös fájlba is íródjanak ki a koherencia értékek.
# TODO: kiíratni a coherence mappába a korpuszhoz használt fájlneveket egy tsv-be.
# TODO: a kiírt fájlba, amiben látszanak a topikok és a dokumentumok, ott a dokumentum szövege ne listaként, hanem stringként íródjon ki + a fájlnév külön oszlopban legyen.

# Minimal article length (words).
minimal_length = 5

# Settings.
c_times = 5  # number of times to be run
c_start = 5  # starting topic number
c_limit = 20  # max. topic number
c_step = 1  # stepping interval

# Outfile path for writing out model coherences and models.
outfile_path = os.path.join('C:/Users/zabor/Desktop/LDA/Data',
                            'C:/Users/zabor/Desktop/LDA/Data/model',
                            'C:/Users/zabor/Desktop/LDA/Data/coherence')

# Mallet settings.
# mallet_path = os.path.join('C:', 'Work', 'mallet', 'mallet-2.0.8', 'bin', 'mallet')
mallet_path = r'C:/mallet/mallet-2.0.8/bin/mallet.bat'
os.environ.update({'MALLET_HOME':r'C:/mallet/mallet-2.0.8/'})

if __name__ == '__main__':
    freeze_support()

    start_time = datetime.now()

    def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):
        """
        Compute c_v coherence for various number of topics

        Parameters:
        ----------
        dictionary : Gensim dictionary
        corpus : Gensim corpus
        texts : List of input texts
        limit : Max num of topics

        Returns:
        -------
        model_list : List of LDA topic models
        coherence_values : Coherence values corresponding to the LDA model with respective number of topics
        """
        coherence_values = []
        model_list = []
        rs = 0
        for num_topics in tqdm.tqdm(range(start, limit, step), leave=False):
            model = gensim.models.wrappers.LdaMallet(mallet_path,
                                                     corpus=corpus,
                                                     num_topics=num_topics,
                                                     workers=3,
                                                     id2word=dictionary,
                                                     optimize_interval=10,
                                                     #rs+run_number=13 ha t18_r01 a modell
                                                     random_seed=rs+run_number)
            rs += 1
            model_list.append(model)
            coherencemodel = CoherenceModel(model=model,
                                            texts=texts,
                                            dictionary=dictionary,
                                            processes=1,
                                            coherence='c_v')
            coherence_value = coherencemodel.get_coherence()
            coherence_values.append(coherence_value)

            # Save model.
            model.save(os.path.join(outfile_path, 'model_topics_t'+str("{:02d}".format(num_topics))+'_r'+str("{:02d}".format(run_number+1))+'.model'))

            # Save visualization.
            if save_visualization:
                lda = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(model)
                vis = gensimvis.prepare(lda, corpus, dictionary, sort_topics=False)
                pyLDAvis.save_html(vis, os.path.join(outfile_path, 'visualization_t'+str("{:02d}".format(num_topics))+'_r'+str("{:02d}".format(run_number+1))+'.html'))

            # Save document to topic Excel.
            if save_document_to_topic:
                df_topic_sents_keywords = format_topics_sentences(ldamodel=model, corpus=corpus, texts=original_documents)
                df_dominant_topic = df_topic_sents_keywords.reset_index()
                df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
                df_dominant_topic.to_excel(os.path.join(outfile_path, 'document_dominant_topics_t'+str("{:02d}".format(num_topics))+'_r'+str("{:02d}".format(run_number+1))+'.xlsx'))

            print('\nCurrent model coherence is '+str(coherence_value)+' with '+str(num_topics)+' topics.')
        return model_list, coherence_values


    def format_topics_sentences(ldamodel, corpus, texts):
        # Init output
        sent_topics_df = pd.DataFrame()

        # Get main topic in each document
        for i, row in enumerate(ldamodel[corpus]):
            row = sorted(row, key=lambda x: (x[1]), reverse=True)
            # Get the Dominant topic, Perc Contribution and Keywords for each document
            for j, (topic_num, prop_topic) in enumerate(row):
                if j == 0:  # => dominant topic
                    wp = ldamodel.show_topic(topic_num)
                    topic_keywords = ", ".join([word for word, prop in wp])
                    sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_keywords]), ignore_index=True)
                else:
                    break
        sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

        # Add original text to the end of the output
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
        return sent_topics_df

    # Load corpus files.
    in_path = 'C:/Users/zabor/Desktop/LDA/Data/txts_contwrd'
    in_files = [f for f in os.listdir(in_path) if os.path.isfile(os.path.join(in_path, f))]

    # Add all files to a list.
    txt = []
    files_used = []
    for filename in tqdm.tqdm(in_files):
        with open(os.path.join(in_path, filename), 'r', encoding='utf-8') as infile:
            article = infile.read()
        if len(article.split(' ')) > minimal_length:
            txt.append(article)
            files_used.append(filename)

    # Create lists containing the texts and the words.
    txts = []
    for t in txt:
        txts.append(t.strip().split())
    words = list(itertools.chain(*txts))

    # Load original documents.
    original_documents_folder = os.path.join(in_folder_original)
    original_document_files = [f for f in os.listdir(original_documents_folder) if os.path.isfile(os.path.join(original_documents_folder, f))]

    # Add all files to a list.
    original_documents = []
    for filename in tqdm.tqdm(original_document_files):
        if filename in files_used:
            with open(os.path.join(original_documents_folder, filename), 'r', encoding='utf-8') as infile:
                article = infile.read()
            lst = [str(filename + ': ' + article)]
            original_documents.append(lst)

    # Create Dictionary
    dictionary = corpora.Dictionary(txts)
    dictionary.save('dictionary.dict')

    # Create corpus
    corpus = [dictionary.doc2bow(text) for text in txts]

    # Write corpus in Mallet format to disk
    MalletCorpus.serialize("corpus.mallet", corpus, dictionary)

    # Read corpus
    # corpus = MalletCorpus("data/corpus/corpus.mallet")

    # Run models.
    for run_number in tqdm.tqdm(range(0, c_times)):
        print('Starting run number: '+str("{:02d}".format(run_number+1))+'.')
        model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=txts, start=c_start, limit=c_limit, step=c_step)

        with open(os.path.join(outfile_path, 'coherence_values_r'+str("{:02d}".format(run_number+1))+'.tsv'), 'w', encoding='utf-8') as outfile:
            c = 0
            for value in coherence_values:
                outfile.write(str(model_list[c].num_topics)+'\t'+str(value)+'\n')
                c += 1

        # Create plot.
        x = range(c_start, c_limit, c_step)
        plt.plot(x, coherence_values)
        plt.xlabel("Num Topics")
        plt.ylabel("Coherence score")
        plt.savefig(os.path.join(outfile_path, 'coherence_values_r'+str("{:02d}".format(run_number+1))+'.png'))
        # plt.show()

    end_time = datetime.now()
    print('All finished.\nDuration: {}'.format(end_time - start_time))

In [ ]:
model = gensim.models.LdaModel.load('C:/Users/zabor/Desktop/LDA/Data/model/coherence/model_topics_t18_r02.model')
#optimális modell betöltése

In [ ]:
model_lda = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(model)
vis = gensimvis.prepare(model_lda, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(vis, 'C:/Users/zabor/Desktop/LDA/Data/model_topics_t18_r02.html')
# LDA model vizualizáció kimentése (HTML-ben)